In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import os
from datetime import datetime
#from datetime import date
import pymysql
from sqlalchemy import create_engine
from decouple import config 
from dotenv import load_dotenv
from pandas import (
    read_sql_query, 
    DataFrame,
    ExcelWriter,
    options
)
from dataframe_image import export


from enum import Enum
import openpyxl
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


from IPython.display import display_html


In [2]:
class Period(Enum):
    start = '2022-04-30'
    end = datetime.today().strftime("%Y-%m-%d")
    date_du_jour = datetime.today().strftime("%d_%m_%Y")

In [3]:
load_dotenv()
# get the environment variables needed
USER = config('USRCaris')
PASSWORD = config('PASSCaris')
HOSTNAME = config('HOSTCaris')
DBNAME = config('DBCaris')

# get the engine to connect and fetch
engine = create_engine(
    f"mysql+pymysql://{USER}:{PASSWORD}@{HOSTNAME}/{DBNAME}")



query = f'''
SELECT 
    dh.id_patient,
    GROUP_CONCAT(DISTINCT dhs.id_group, '') AS id_group,
    dh.created_by,
    au.email AS email,
    au.username,
    MAX(dh.created_at) AS entry_date,
    COUNT(*) AS quantity
FROM
    caris_db.dream_hivinfos dh
        LEFT JOIN
    auth_users au ON au.id = dh.created_by
        LEFT JOIN
    dream_hivinfos_session dhs ON dhs.id = dh.id_dream_hivinfos_session
GROUP BY  dh.created_by, dh.id_patient 
HAVING (entry_date >= '{Period.start.value}'
    AND entry_date <= '{Period.end.value}')
'''




dhi1 = read_sql_query(query, engine, parse_dates=True)


# close the pool of connection
engine.dispose()

True

In [4]:
dhi1.id_patient.count()
dhi =  dhi1.groupby(['username']).count()
dhi.quantity.sum()

1429

1429

In [5]:
dhi1.columns
dhi

Index(['id_patient', 'id_group', 'created_by', 'email', 'username',
       'entry_date', 'quantity'],
      dtype='object')

,id_patient,id_group,created_by,email,entry_date,quantity
username,,,,,,
Armando,84,84,84,84,84,84
BonWoodRidchard,166,166,166,166,166,166
Ernstbelabre,124,124,124,124,124,124
France14,21,21,21,21,21,21
billyjolicoeur,174,174,174,174,174,174
dchelot,228,228,228,228,228,228
fbyron,530,530,530,530,530,530
pggd1234,102,102,102,102,102,102


In [6]:
dhi_report = dhi[['quantity']]
dhi_report.reset_index(inplace=True)
dhi_report

,username,quantity
0,Armando,84
1,BonWoodRidchard,166
2,Ernstbelabre,124
3,France14,21
4,billyjolicoeur,174
5,dchelot,228
6,fbyron,530
7,pggd1234,102


In [7]:
dhi_report.to_excel('rapport_hivinfos.xlsx',na_rep="",index=False)